In [59]:
#Tesla stock data:https://finance.yahoo.com/quote/TSLA/history?period1=1612137600&period2=1614038400&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true
#Tesla Tweeter data : 01.02 -23.02 

import pandas as pd

df = pd.read_excel (r'/Users/mitarbeiter/datascience_project/code_op_solo_project/Tesla.Xlsx', sheet_name='Sheet0')
Tesla = pd.read_csv(r'/Users/mitarbeiter/datascience_project/code_op_solo_project/TSLA.csv')
# pd.options.display.max_rows = 4000

In [60]:
#!pip install autopep8

In [61]:
data=df[['published','document_content']]


In [62]:
df['published']= pd.to_datetime(df.published)

df['published'] = df['published'].dt.strftime('%Y-%m-%d')

df['published'].head()

data=df[['published','document_content']]
data

,published,document_content
0,2021-02-01,@TSLAFanMtl That's what every shit company say...
1,2021-02-01,What skills are needed to earn $100k+ as a Con...
2,2021-02-01,.@CBS_Herridge .@nytpolitics .@alivitali .@Mar...
3,2021-02-01,Can I have a Tesla
4,2021-02-01,Everyone who put a $100 deposit on the Tesla t...
...,...,...
501388,2021-02-23,@Saweetie @ghxttowitch purring in our Tesla’s ...
501389,2021-02-23,#stocks helping to tank the high-flying #ARK I...
501390,2021-02-23,Tesla is up 185x in the 11 years since I first...
501391,2021-02-23,"metaphysicists rather than scientists."" ~ Niko..."


In [64]:
#==========================================
##Tesla Stock Data
#==========================================

In [66]:
# Tesla=Tesla.drop(['Twitter_mention'], axis=1)
# Tesla.head(2)

In [67]:
#how many mentions there are 

data_mention_count=data[['publish','document_content']]
data_mention_count.rename(columns={'Date':'Date'},inplace=True)
twitter_mention_count=data_mention_count.groupby(by=["Date"]).count()
# twitter_mention_count

Tesla=pd.merge(Tesla, twitter_mention_count,  on='Date', how='inner')




KeyError: "['Date'] not in index"

In [40]:
Tesla.rename(columns={'document_content':'twitter_volume'},inplace=True)
Tesla.head(3)

,Date,Open,High,Low,Close,Adj Close,Volume,twitter_volume
0,2021-02-01,814.289978,842.000000,795.559998,839.809998,839.809998,25391400,18540
1,2021-02-02,844.679993,880.500000,842.200012,872.789978,872.789978,24346200,17737
2,2021-02-03,877.020020,878.080017,853.059998,854.690002,854.690002,18343500,17114


In [ ]:
stock_data=Tesla[['Date','Close','Volume']]
stock_diff=stock_data.set_index('Date').diff()
stock_diff
Tesla=pd.merge(Tesla, stock_diff,  on='Date', how='inner')
Tesla.head()
Tesla.rename(columns={'Close_x':'Close','Volume_x':'Volume','Close_y':'Close_diff','Volume_y':'Volume_diff'},inplace=True)

In [ ]:
Tesla.rename(columns={'Close_x': 'Close', 'Volume_x': 'Volume', 'Close_y':'Close_diff', 'Volume_y':'Volume_diff'}, inplace=True)
Tesla.head()

In [ ]:
#***Plotting*** - #I still need to add daily average sentiment :) 
# but the foramt is the same so if it works I can just add one additional column

import pandas as pd
import numpy as np

from bokeh.io import show, output_notebook, curdoc
from bokeh.plotting import figure
from bokeh.layouts import row, column
from bokeh.resources import INLINE

line_chart = Tesla(plot_width=1000, plot_height=400, x_axis_type="Date",
                    title="Tesla stock data from 01.02 to 23.02")

line_chart.line(
        x="Date", y="Close",
        line_width=0.5, line_color="red",
        legend_label = "Close",
        source=Tesla_for_widget
        )

line_chart.xaxis.axis_label = 'Time'
line_chart.yaxis.axis_label = 'Price ($)'

line_chart.legend.location = "top_left"

show(line_chart)

from bokeh.models import CheckboxButtonGroup

checkbox_options = ['Open','High','Low','Close','Adj Close','Volume','Close_diff','Volume_diff']

checkbox_grp = CheckboxButtonGroup(labels=checkbox_options, active=[0], button_type="success")
show(checkbox_grp)


In [43]:
#testing plot widget

import pandas as pd
import numpy as np
import seaborn as sns
import cufflinks as cf
cf.go_offline()

In [ ]:
data.head()

In [44]:
from pathlib import Path
from typing import List

import nltk
import numpy as np
import pandas as pd
import re
import string
from matplotlib import pyplot as plt

# Tokenizers libs
from nltk.tokenize import sent_tokenize, TweetTokenizer
from nltk import word_tokenize, ne_chunk, ngrams, pos_tag
from nltk.corpus import movie_reviews, stopwords
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer


from sklearn.feature_extraction.text import CountVectorizer

# Global Parameters
stop_words = set(stopwords.words('english'))

In [14]:
# # Install Libraries
# !pip install textblob
# !pip install tweepy
# !pip install pycountry
# !pip install wordcloud
# !pip install langdetect

In [ ]:
def preprocess_tweet_text(tweet):

    tweet= tweet.lower()
    # Remove urls
    tweet = re.sub(r"http\S+|www\S+|https\S+\?+", '', str(tweet), flags=re.MULTILINE)
    # Remove user @ references and '#' from tweet
    tweet = re.sub(r'[^a-zA-z0-9.,!?/:;\"\'\s\.\,\!\?\*]' ,'', str(tweet))
    # Remove punctuations
#     tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords
    tknz = TweetTokenizer()
    tweet_tokens = tknz.tokenize(str(tweet))
    filtered_words = [w for w in tweet_tokens if not w in stop_words]
 
    #ps = PorterStemmer()
    #stemmed_words = [ps.stem(w) for w in filtered_words]
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in filtered_words]
 
    #count most frequent words
#     import nltk
# #     allWords = tknz.tokenize(str(data.document_content))
#     all_words=str(lemma_words)
#     allWordDist = nltk.FreqDist(w.lower() for w in all_words)
#     stopwords = nltk.corpus.stopwords.words('english')
#     allWordExceptStopDist = nltk.FreqDist(w.lower() for w in all_words if w not in stopwords)  
#     mostCommon= allWordDist.most_common(30)
#     print(mostCommon)
    
#     return " ".(lemma_words)
    return lemma_words

In [ ]:
from IPython.display import display
pd.options.display.max_rows = None

In [ ]:
data['token']=data['document_content'].apply(preprocess_tweet_text)

In [ ]:
data.head(5)

In [ ]:
# import pandas as pd
# import numpy as np

# from bokeh.io import show, output_notebook, curdoc
# from bokeh.plotting import figure
# from bokeh.layouts import row, column
# from bokeh.resources import INLINE

In [ ]:
# sudo pip install ipywidgets
# jupyter nbextension enable --py widgetsnbextension

In [ ]:
# from ipywidgets import interact,interactive,fixed,interact_manual
# import ipywidgets as widgets

In [ ]:
# import ipywidgets as widgets
# from ipywidgets import interact, interactive, fixed, interact_manual

# drop_down=widgets.Dropdown(
#     options=data.columns.values,
#     value='published',
#     description='Column:',
#     disabled=False,
# )

# def update_barchart(columns):
#     return data.groupby(columns).size().plot(kind='bar',stacked=True)

# interactive(update_barchart, columns=drop_down)

In [168]:
# def mostCommonWord(self, paragraph: str, banned: List[str]) -> str:
#         #1). replace the punctuations with spaces,
#         #      and put all letters in lower case
#         normalized_str = ''.join([c.lower() if c.isalnum() else ' ' for c in paragraph])

#         #2). split the string into words
#         words = normalized_str.split()

#         word_count = defaultdict(int)
#         banned_words = set(banned)

#         #3). count the appearance of each word, excluding the banned words
#         for word in words:
#             if word not in banned_words:
#                 word_count[word] += 1

#         #4). return the word with the highest frequency
#         return max(word_count.items(), key=operator.itemgetter(1))[0]

In [ ]:
**************************** plotting ***********************

In [126]:
# !conda install -y nodejs
# !pip install --upgrade jupyterlab
# !pip labextension install @jupyter-widgets/jupyterlab-manager
# !pip labextension install jupyter-matplotlib
# !pip nbextension enable --py widgetsnbextension

In [107]:
# data['token'].astype(str)
#daily average compound 


In [46]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
# from textblob import TextBlob

In [45]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

df['compound'] = [analyser.polarity_scores(x)['compound'] for x in data['document_content']]
df['neg'] = [analyser.polarity_scores(x)['neg'] for x in data['document_content']]
df['neu'] = [analyser.polarity_scores(x)['neu'] for x in data['document_content']]
df['pos'] = [analyser.polarity_scores(x)['pos'] for x in data['document_content']]

In [47]:
doc=df[['published','document_content','authors_name','sentiment_display','compound']]

In [48]:
##let's plot this sentiment##
doc.head()
doc.rename(columns={'published':'Date'},inplace=True)

In [49]:
sentiment_avg=doc.groupby(['Date'])['compound'].mean()
sentiment_avg=pd.DataFrame(sentiment_avg)
# Tesla=Tesla.drop(columns=['sentiment_avg', 'sentiment_avg_'])

Tesla_sentiment=pd.merge(Tesla,sentiment_avg,how='inner',on='Date')
Tesla_sentiment
Tesla_sentiment.rename(columns={'compound':"daily_sentiment_avg"},inplace=True)
Tesla_sentiment

Tesla=Tesla_sentiment['daily_sentiment_avg']*3


In [50]:
***plotting***


import pandas as pd
import numpy as np

from bokeh.io import show, output_notebook, curdoc
from bokeh.plotting import figure
from bokeh.layouts import row, column
from bokeh.resources import INLINE


SyntaxError: invalid syntax (<ipython-input-50-91156642277c>, line 1)

In [ ]:
# counts = df['sentiment_display'].value_counts().to_dict()
# print(counts)
#( 1: 203516, -1: 38449}
# df_daily_avg = df['compound'].resample('d').mean()
twitter_mention_count=data_mention_count.groupby(by=["Date"]).count()
# twitter_mention_count
Tesla_sentiment

# pd.concat(twitter_mention_count,Tesla_sentiment,ignore_index=True)



In [ ]:
twitter_mention_count['Date']=['2021-02-01','2021-02-02','2021-02-03','2021-02-04','2021-02-05','2021-02-06','2021-02-07','2021-02-08','2021-02-09','2021-02-10','2021-02-11','2021-02-12','2021-02-13','2021-02-14','2021-02-15','2021-02-16','2021-02-17','2021-02-18','2021-02-19','2021-02-20','2021-02-21','2021-02-22','2021-02-23']
# twitter_mention_count

In [ ]:
# Matplotlib pyplot provides plotting API
import matplotlib as mpl
from matplotlib import pyplot as plt
# For output plots inline in notebook:
%matplotlib inline
# For interactive plot controls on MatplotLib output:
# %matplotlib notebook
# Set the default figure size for all inline plots
# (note: needs to be AFTER the %matplotlib magic)
plt.rcParams['figure.figsize'] = [8, 5]

In [ ]:
# # Show the top 20 users in a bar plot with Matplotlib.
# top_n = 23
# # Create the bars on the plot
# plt.bar(x=range(top_n), # start off with the xticks as numbers 0:19
#         height=twitter_mention_count[0:top_n]['document_content'])
# # Change the xticks to the correct user ids
# plt.xticks(range(top_n), twitter_mention_count[0:top_n]['Date'], 
#            rotation=60)
# # Set up the x, y labels, titles, and linestyles etc.
# plt.ylabel("Twitter Mentions")
# plt.xlabel("Day")
# plt.title("Daily Twitter Mention")
# plt.gca().yaxis.grid(linestyle=':')

In [ ]:
# Creating a bar plot with seaborn
import seaborn as sns
sns.set()
sns.barplot(
    x='Date', 
    y='document_content', 
    color='salmon', 
    data=twitter_mention_count[0:23]
)
# Again, Matplotlib style formatting commands are used
# to customise the output details.
plt.xticks(rotation=60)
plt.ylabel("Number of mentions")
plt.xlabel("Day")
plt.title("Number of Mentions")
plt.gca().yaxis.grid(linestyle=':')

In [ ]:
# !conda install -c plotly plotly --yes
# !conda install -c conda-forge cufflinks-py --yes


In [ ]:
# !pip install plotly --upgrade
# !pip install cufflinks --upgrade

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
import cufflinks as cf
# Going offline means you plot only locally, and dont need a plotly username / password
cf.go_offline()
# Create an interactive bar chart:
twitter_mention_count[0:20].iplot(
    x='Date',
    y='document_content',
    kind='bar'
)

In [ ]:
#overall sentiment 

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
# if using a Jupyter notebook, include:
%matplotlib inline

# Pie chart
labels = ['positive','negative']
sizes = [203516,38449]
# only "explode" the 2nd slice (i.e. 'Hogs')
explode = (0, 0.1,)  
fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
# Equal aspect ratio ensures that pie is drawn as a circle
ax1.axis('equal')  
plt.tight_layout()
plt.show()

In [ ]:
**** wordcloud**** 

In [ ]:
#what is the most frequent words?

data['document_content']=data['document_content'].apply(lambda x:" ".join(x.lower() for x in x.split()))
data['document_content']=data['document_content'].str.replace('[^\w\s]',' ')
data['document_content']=data['document_content'].str.replace('\d',' ')

#stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw=stopwords.words('english')
data['document_content']=data['document_content'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

#lemma
from textblob import Word
nltk.download('wordnet')
data['document_content']=data['document_content'].apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))

#punctuation
data['document_content']=data['document_content'].str.replace("rt"," ")

data['document_content'].head()

In [ ]:
=======wordcloud is not showing==========

In [ ]:
#wordcloud is not showing 

from os import path
from PIL import Image
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
import matplotlib.pyplot as plt

text="".join(i for i in data.document_content)

wc=WordCloud(background_color="black").generate(str(data.document_content))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.tight_layout(pad=1)
plt.show()


In [ ]:
##named entiity recognition
# !pip install - U spacy
# !python -m spacy download en_core_web_sm
# !pip install scispacy

In [ ]:
# !python -m spacy download en_core_web_lg
# !python -m spacy download en_core_web_sm

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [ ]:
df=df
tex=df.document_content

In [ ]:
nlp=en_core_sci_sm.load()
doc=nlp(text)
displacy_image=displacy,render(doc,jupyeter=True, style='ent')

In [71]:
# #Sentiment Distribution pie chart

# from textblob import TextBlob
# def sentiment_a(df):
#     text=data['document_content']
#     for i in range(0,len(text)):
#         textB=TextBlob(text[i])
#         sentiment_a=textB.sentiment.polarity
#         df.at[i,"sentiment_twitter"]=sentiment_a
        
#         if sentiment_a < 0.00:
#             sentiment= "negative"
#             df.at[i,"sentiment"]=sentiment
            
#         elif sentiment_a> 0.45:
#             sentiemnt= "positive"
#             df.at[i,"sentiment"]=sentiment
            
#         else:
#             sentiment="netural"
#             df.at[i,"sentiment"]=sentiment
#     return sentiment_a
            
            

In [73]:
# sentiment_a(data['document_content'])

In [ ]:
#spacy -> to find the most mentioned words 

In [ ]:
# nlp = spacy.load(“en_core_web_lg”)
# doc = nlp(“At midnight the doorbell rang, startling him fearfully.”)

In [16]:
#========let's plot the sentiment================

In [ ]:
# show(scatter)

In [ ]:
# %matplotlib widget

In [ ]:
# from ipywidgets import interact, interact_manual

In [ ]:
# new_df=data[['published','compound']]

In [ ]:
# df = df.set_index('Date').groupby(pd.Grouper(freq='d')).mean().dropna(how='all')

In [ ]:
# #I want to simplify it 
# def sentiment_analyzer_scores(text):
#     score = analyser.polarity_scores(text)
#     lb = score['compound']
#     if lb >= 0.5:
#         return 1
#     elif (lb > -0.5) and (lb < 0.5):
#         return 0
#     else:
#         return -1

In [ ]:
# sentiment_analyzer_scores(data['document_content'])

In [ ]:
# ! pip install -U spacy

In [175]:
# !pip install yellowbrick

In [ ]:
pd.Series(data.token).value_counts().sort_values(ascending=True)

In [165]:
# # def preprocess_tweet_text(tweet):

# # #     tweet= tweet.lower()
# #     # Remove urls
# #     tweet = re.sub(r"http\S+|www\S+|https\S+\?+", '', str(tweet), flags=re.MULTILINE)
# #     # Remove user @ references and '#' from tweet
# #     tweet = re.sub(r'[^a-zA-z0-9.,!?/:;\"\'\s\.\,\!\?\*]' ,'', str(tweet))
# #     # Remove punctuations
# # #     tweet = tweet.translate(str.maketrans('', '', string.punctuation))
# #     # Remove stopwords
# #     tknz = TweetTokenizer()
# #     tweet_tokens = tknz.tokenize(str(tweet))
# #     filtered_words = [w for w in tweet_tokens if not w in stop_words]
 
# #     #ps = PorterStemmer()
# #     #stemmed_words = [ps.stem(w) for w in filtered_words]
# #     lemmatizer = WordNetLemmatizer()
# #     lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in filtered_words]
 
# def most_frequent(tweet):
#     tknz = TweetTokenizer()
#     import nltk
#     tweet = re.sub(r'[^a-zA-z0-9.,!?/:;\"\'\s\.\,\!\?\*\:\]' ,'', str(tweet))
#     tweet = re.sub(r"http\S+|www\S+|https\S+\?+", '', str(tweet), flags=re.MULTILINE)
#     allWords = tknz.tokenize(str(tweet))
#     all_words=str(allWords)
#     allWordDist = nltk.FreqDist(w.lower() for w in all_words)
#     stopwords = nltk.corpus.stopwords.words('english')
#     allWordExceptStopDist = nltk.FreqDist(w.lower() for w in all_words if w not in stopwords)  
#     mostCommon= allWordDist.most_common(50)
#     print(mostCommon)

In [ ]:
# #most frequent words
# frequent=most_frequent(data.document_content)

In [38]:
# !pip install WorldCloud
# from word_cloud.word_cloud_generator import WordCloud
# from IPython.core.display import HTML

ERROR: Could not find a version that satisfies the requirement WorldCloud (from versions: none)
ERROR: No matching distribution found for WorldCloud


ModuleNotFoundError: No module named 'word_cloud'

In [ ]:
#1. most often mentioned words
#2. word cloud - positive
#3. word cloud- negative
#4. word cloud- netural 

In [ ]:
# print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
# print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# # Find named entities, phrases and concepts
# for entity in doc.ents:
#     print(entity.text, entity.label_)

In [ ]:
#entity recognition

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [ ]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [ ]:
sent = preprocess(ex)
sent

In [ ]:
pattern = 'NP: {<DT>?<JJ>*<NN>}'

In [ ]:
cp = nltk.RegexpParser(pattern)
cs = cp.parse(sent)
print(cs)

In [ ]:
from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint
iob_tagged = tree2conlltags(cs)
pprint(iob_tagged)